## Notebook for training a DGAT model

This notebook is aimed at training a DGAT model with all datasets we used in the paper.

Please replace the **sample_list**, **model_save_dir**, **pyg_save_dir** and **dataset_save_dir** with actual paths.

In [1]:
import sys
import numpy as np

import scanpy as sc
import torch
import warnings
warnings.filterwarnings('ignore')

from utils.Preprocessing import qc_control_cytassist, normalize, clean_protein_names, preprocess_train_list
from Model.Train_and_Predict import train

import random
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

sample_list = ['Tonsil', 'Tonsil_AddOns', 'Breast', 'Glioblastoma','PBC-PR_6835-5A','PBC_PR_6837']

dataset_save_dir = './DGAT_training_datasets'

pyg_save_dir = './pyg_data' # Any dir that you want. Building a graph might take 3 mins or more (depends on the scale of the dataset).This directory is used to save the graph data in PyG format, which can be reused for training.

model_save_dir = './DGAT_pretrained_models'# Directory to save the trained DGAT model.

os.makedirs(dataset_save_dir, exist_ok=True)
os.makedirs(dataset_save_dir, exist_ok=True)
os.makedirs(model_save_dir, exist_ok=True)

### Loading dataset
You can replace this with your own dataset. The dataset should be in the format of AnnData object with RNA and protein data.

In [2]:
adata_list = []
pdata_list = []
for sample in sample_list:
    print(sample)
    adata = sc.read_h5ad(f'{dataset_save_dir}/{sample}_RNA.h5ad')
    pdata = sc.read_h5ad(f'{dataset_save_dir}/{sample}_ADT.h5ad')
    adata_list.append(adata)
    pdata_list.append(pdata)


Tonsil
Tonsil_AddOns
Breast
Glioblastoma
PBC-PR_6835-5A
PBC_PR_6837


### Preprocessing
In this step, we will perform quality control, normalization, and finding the common genes and proteins across the datasets.


In [3]:
# Performs quality control and normalization on the training datasets
common_gene, common_protein = preprocess_train_list(adata_list, pdata_list)

Common genes before QC: 18085
Common proteins before QC: 35
Num of encoding genes: 30
Common genes after QC: 11535
Common proteins after QC: 31
Common gene names saved to common_gene_11535.txt
Common protein names saved to common_protein_31.txt


In [4]:
adata_list = [adata[:,common_gene] for adata in adata_list]
pdata_list = [pdata[:,common_protein] for pdata in pdata_list]

### Training


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Trains the DGAT model using the preprocessed training datasets. It takes in the list of processed ST datasets and protein datasets, and the directory to save the processed graph data (for more efficient reproduction).
model_components = train(adata_list, pdata_list, pyg_save_dir)

model_save_path = f"{model_save_dir}/{len(common_gene)}_gene_{len(common_protein)}_protein"
print(model_save_path)
os.makedirs(model_save_path, exist_ok=True)
torch.save(model_components['mRNA_encoder'].state_dict(), model_save_path+f'/encoder_mRNA.pth')
torch.save(model_components['mRNA_decoder'].state_dict(), model_save_path+f'/decoder_mRNA.pth')
torch.save(model_components['protein_encoder'].state_dict(), model_save_path+f'/encoder_protein.pth')
torch.save(model_components['protein_decoder'].state_dict(), model_save_path+f'/decoder_protein.pth')



Common genes: 11535
Common proteins: 31
Creating or loading dataset
Loading preprocessed data for sample 'CytAssist_FFPE_Protein_Expression_Human_Tonsil' from '/ix/hosmanbeyoglu/haw309/pyg_data/CytAssist_FFPE_Protein_Expression_Human_Tonsil_11535_new_spatial=6_1hop_new_filtering.pth'
Loading preprocessed data for sample 'CytAssist_FFPE_Protein_Expression_Human_Tonsil_AddOns' from '/ix/hosmanbeyoglu/haw309/pyg_data/CytAssist_FFPE_Protein_Expression_Human_Tonsil_AddOns_11535_new_spatial=6_1hop_new_filtering.pth'
Loading preprocessed data for sample 'CytAssist_FFPE_Protein_Expression_Human_Breast_Cancer' from '/ix/hosmanbeyoglu/haw309/pyg_data/CytAssist_FFPE_Protein_Expression_Human_Breast_Cancer_11535_new_spatial=6_1hop_new_filtering.pth'
Loading preprocessed data for sample 'CytAssist_FFPE_Protein_Expression_Human_Glioblastoma' from '/ix/hosmanbeyoglu/haw309/pyg_data/CytAssist_FFPE_Protein_Expression_Human_Glioblastoma_11535_new_spatial=6_1hop_new_filtering.pth'
Loading preprocessed dat

Epoch [1/100] Total Loss: 10.0399 | mRNA Recon: 1.1506 | Protein Recon: 0.2508 | Alignment: 0.4901 | Protein Pred: 0.2494 | mRNA Pred: 1.1538
Epoch 1: EB evidence = 0.1741


Epoch [2/100] Total Loss: 8.0588 | mRNA Recon: 1.0262 | Protein Recon: 0.2789 | Alignment: 0.3883 | Protein Pred: 0.2759 | mRNA Pred: 1.0269
Epoch 2: EB evidence = 1.0062


Epoch [3/100] Total Loss: 7.7179 | mRNA Recon: 1.0160 | Protein Recon: 0.2753 | Alignment: 0.3214 | Protein Pred: 0.2716 | mRNA Pred: 1.0168
Epoch 3: EB evidence = 0.5200


Epoch [4/100] Total Loss: 7.5451 | mRNA Recon: 1.0116 | Protein Recon: 0.3561 | Alignment: 0.2750 | Protein Pred: 0.3536 | mRNA Pred: 1.0124
Epoch 4: EB evidence = 0.4994


...

Epoch [71/100] Total Loss: 6.3493 | mRNA Recon: 0.9956 | Protein Recon: 0.0856 | Alignment: 0.0548 | Protein Pred: 0.1113 | mRNA Pred: 0.9985
Epoch 71: EB evidence = 0.9734


Epoch [72/100] Total Loss: 6.3438 | mRNA Recon: 0.9907 | Protein Recon: 0.1336 | Alignment: 0.0582 | Protein Pred: 0.1692 | mRNA Pred: 0.9957
Epoch 72: EB evidence = 0.9711


Epoch [73/100] Total Loss: 6.3422 | mRNA Recon: 0.9457 | Protein Recon: 0.0912 | Alignment: 0.0623 | Protein Pred: 0.1224 | mRNA Pred: 0.9532
Epoch 73: EB evidence = 0.9707


Epoch [74/100] Total Loss: 6.3382 | mRNA Recon: 0.9676 | Protein Recon: 0.0818 | Alignment: 0.0600 | Protein Pred: 0.1082 | mRNA Pred: 0.9740
Epoch 74: EB evidence = 0.9804


Epoch [75/100] Total Loss: 6.3340 | mRNA Recon: 0.9567 | Protein Recon: 0.0923 | Alignment: 0.0633 | Protein Pred: 0.1173 | mRNA Pred: 0.9605
Epoch 75: EB evidence = 0.9796
--> EB early stopping at epoch 75 (mean_evidence=0.9628, threshold=0.96)
/ix/hosmanbeyoglu/haw309/DGAT_models/11535_gene_31_protein
